In [7]:
from osgeo import gdal, ogr, osr
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
from IPython.display import display, HTML
display(HTML('<style>.container { width:100% !important; }</style>'))
plt.rc('figure', figsize=(32,24))
sns.set_context("poster")
sns.set_style("whitegrid")

Data source:
https://data.cityofnewyork.us/City-Government/1-foot-Digital-Elevation-Model-DEM-/dpc8-z3jc

In [16]:
dataset = gdal.Open('data/DEM_LiDAR_1ft_2010_Improved_NYC.img', gdal.GA_ReadOnly)

In [17]:
dataset.RasterCount

1

In [18]:
dataset.GetProjectionRef()

'PROJCS["NAD83 / New York Long Island (ftUS)",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",40.1666666666667],PARAMETER["central_meridian",-74],PARAMETER["standard_parallel_1",40.6666666666667],PARAMETER["standard_parallel_2",41.0333333333333],PARAMETER["false_easting",984250],PARAMETER["false_northing",0],UNIT["US survey foot",0.304800609601219,AUTHORITY["EPSG","9003"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'

In [19]:
padfTransform = dataset.GetGeoTransform()
padfTransform

(910719.3, 1.0, 0.0, 275160.675, 0.0, -1.0)

In [24]:
print('Width:\t %i feet' % dataset.RasterXSize)
print('Height:\t %i feet' % dataset.RasterYSize)

Width:	 158100 feet
Height:	 156100 feet


In [25]:
# Create arrays of X and Y state plabne coordinates using dataset geo transform
Xp_arr = padfTransform[0] + np.arange(0, dataset.RasterXSize, 1) * padfTransform[1]
Yp_arr = padfTransform[3] + np.arange(0, dataset.RasterYSize, 1) * padfTransform[5]

In [ ]:
# Spatial Reference System for state plane to lat/lon coordinate transform
inputEPSG = 2263
outputEPSG = 4140

point = ogr.Geometry(ogr.wkbPoint)

inSpatialRef = osr.SpatialReference()
inSpatialRef.ImportFromEPSG(inputEPSG)

outSpatialRef = osr.SpatialReference()
outSpatialRef.ImportFromEPSG(outputEPSG)

coordTransform = osr.CoordinateTransformation(inSpatialRef, outSpatialRef)

In [28]:
# Create arrays for lat and lon
lat_arr = []
lon_arr = []
for idx, (Xp, Yp) in enumerate(zip(Xp_arr, Yp_arr)):
    point.AddPoint(Xp, Yp)
    point.Transform(coordTransform)
    lat = point.GetX()
    lon = point.GetY()
    lat_arr.append(lat)
    lon_arr.append(lon)